# Atom feed of highly tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least five times as atom feed, grouped by subject area.

In [6]:
import sys
sys.path.append('../')

import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator

In [7]:
subject_areas = ['covid', 'microbiology', 'neuroscience', 'molecular-biology', 'bioinformatics', 'genomics', 'genetics', 'developmental-biology', 'plant-biology', 'evolutionary-biology', 'ecology', 'immunology', 'cancer-biology', 'cell-biology', 'biophysics']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
subsets['microbiology'] = preprints[(preprints['subject-area'] == 'Microbiology') & (preprints['covid'] == False)]
subsets['neuroscience'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
subsets['molecular-biology'] = preprints[(preprints['subject-area'] == 'Molecular Biology') & (preprints['covid'] == False)]
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
subsets['genomics'] = preprints[(preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') & (preprints['covid'] == False)]
subsets['developmental-biology'] = preprints[(preprints['subject-area'] == 'Developmental Biology') & (preprints['covid'] == False)]
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['immunology'] = preprints[(preprints['subject-area'] == 'Immunology') & (preprints['covid'] == False)]
subsets['cancer-biology'] = preprints[(preprints['subject-area'] == 'Cancer Biology') & (preprints['covid'] == False)]
subsets['cell-biology'] = preprints[(preprints['subject-area'] == 'Cell Biology') & (preprints['covid'] == False)]
subsets['biophysics'] = preprints[(preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
subsets['cell-biology'].head(50)

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
52,10.1101/2021.05.12.443921,22,bioRxiv,Cell Biology,False,Live-Cell Imaging of (p)ppGpp with RNA-based F...,"[{'name': 'Zhining Sun'}, {'name': 'Rigumula W...","<p>Guanosine tetra- and pentaphosphate, (p)ppG...",2021-05-13
119,10.1101/2021.05.12.443932,8,bioRxiv,Cell Biology,False,A Method for Rapid Flow-cytometric Isolation o...,"[{'name': 'Amy L. Kimble'}, {'name': 'Jordan S...",<p>Endothelial cells are important contributor...,2021-05-13
124,10.1101/2021.05.11.443558,8,bioRxiv,Cell Biology,False,Elongation factor ELOF1 drives transcription-c...,"[{'name': 'Marit E Geijer'}, {'name': 'Di Zhou...",<p>Correct transcription is crucial for life. ...,2021-05-11
143,10.1101/2021.05.11.443597,6,bioRxiv,Cell Biology,False,The Drosophila orthologue of the primary cilia...,"[{'name': 'Petra zur Lage'}, {'name': 'Zhiyan ...",<p>Ciliary motility is powered by a suite of h...,2021-05-11
166,10.1101/2021.05.12.443861,5,bioRxiv,Cell Biology,False,Dynein light chain-dependent dimerization of E...,"[{'name': 'Hannah Neiswender'}, {'name': 'Chan...",<p>Egalitarian (Egl) is an RNA adaptor for the...,2021-05-13
171,10.1101/2021.05.12.443936,5,bioRxiv,Cell Biology,False,A Trio of Ubiquitin Ligases Sequentially Drive...,"[{'name': 'Richard S. Marshall'}, {'name': 'Ri...",<p>As central effectors of ubiquitin (Ub)-medi...,2021-05-13


In [8]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular BioRxiv {subject_area.replace("_", " ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')